In [ ]:
!pip install tensorflow_addons

In [ ]:
import pandas as pd
import numpy as np
import cv2
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
import random as rn
import os
import pickle

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , Input ,BatchNormalization, Concatenate, Layer , Maximum , Softmax
from tensorflow.keras.layers import  GlobalAveragePooling2D, AveragePooling2D , Conv1D , MaxPooling1D, Average, Dropout, LayerNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, GRU, ReLU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,confusion_matrix,balanced_accuracy_score
import joblib

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
def load_dataset(ds_name):
  with open(ds_name + '/element_spec', 'rb') as in_:
    es = pickle.load(in_)

  dataset = tf.data.Dataset.load(
      ds_name, es, compression='GZIP'
  )
  return dataset

In [ ]:
def model_1concat_gating(shape1, shape2 , all_list_filters, list_num_experts, same_seed = True, num_classes = 7):

    def set_seed(num_seed):
        SEED = num_seed
        os.environ['PYTHONHASHSEED']=str(SEED)
        np.random.seed(SEED)
        tf.random.set_seed(SEED)
        rn.seed(SEED)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        tf.compat.v1.keras.backend.set_session(sess)

    def expert_block(inputs , num_filters,  num_seed):
        set_seed(num_seed)
        conv = Conv2D(filters = num_filters[0],
                      kernel_size = (3,3) ,
                      activation = tf.nn.relu )(inputs)
        conv = MaxPooling2D(pool_size=(2, 2))(conv)
        conv = BatchNormalization()(conv)
        if  1 <= len(num_filters[1:]) <= 3  :
            for num_filter in num_filters[1:] :
                conv = Conv2D(filters = num_filter,
                              kernel_size = (3,3) ,
                              activation = tf.nn.relu,
                              padding = 'same')(conv)
                conv = MaxPooling2D(pool_size=(2, 2))(conv)
                conv = BatchNormalization()(conv)
        elif len(num_filters[1:]) >= 4 :
            for num_filter in num_filters[1:] :
                conv = Conv2D(filters = num_filter,
                              kernel_size = (3,3) ,
                              activation = tf.nn.relu,
                              padding = 'same')(conv)
                conv = BatchNormalization()(conv)


        #--------------------------------------------------
        conv = Flatten()(conv)
        conv = Dense(units =  1000,
                     activation = tf.nn.relu)(conv)
        conv = BatchNormalization()(conv)
        #-----------------------------
        conv = Dense(units =  600,
                     activation = tf.nn.relu)(conv)
        conv = BatchNormalization()(conv)
        conv = Dropout(rate = 0.5)(conv)
        y_hat = Dense(units = num_classes,
                      activation = None)(conv)
        return y_hat

    def Mixture_of_Experts(inputs,list_num_filters , num_experts, names) :
        assert len(list_num_filters) == num_experts
        num_seed = 0
        expert_outs = []
        for i in range(num_experts):
            if same_seed :
                expert_network = expert_block(inputs, list_num_filters[i], num_seed )
            else :
                expert_network = expert_block(inputs, list_num_filters[i], i )
            expert_outs.append(expert_network)
        expert_concat = tf.keras.layers.Lambda(lambda x: tf.stack(x, axis=1))(expert_outs)
        print('expert concat shape : ', expert_concat.shape)
        # gating network
        gating = Conv2D(filters = 30,
                      kernel_size = (3,3) ,
                      activation = tf.nn.relu )(inputs)
        gating = MaxPooling2D(pool_size=(2, 2))(gating)
        gating = BatchNormalization()(gating)
        gating = Flatten()(gating)
        # ------------------------------------------------
        gating = Dense(800, activation = tf.nn.relu)(gating)
        gating = BatchNormalization()(gating)
        gating = Dropout(rate = 0.5)(gating)
        # -----------------
        gating = Dense(1200, activation = tf.nn.relu)(gating)
        gating = BatchNormalization()(gating)
        gating = Dropout(rate = 0.5)(gating)
        # -----------------
        gating = Dense(num_experts, activation = tf.nn.softmax)(gating)
        gating = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(gating)
        gate_mul_expert = tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x[0] * x[1], axis=1),
                                                 name='gate_mul_expert_' + names )([expert_concat, gating])
        gate_mul_expert = tf.keras.layers.Softmax()(gate_mul_expert)
        print('gate mul expert : ', gate_mul_expert.shape)

        return gate_mul_expert

    assert len(all_list_filters) == list_num_experts
    names = '0'
    set_seed(0)
    #------------------------------------------
    input = Input(shape = (shape1, shape2, 1))
    output = Mixture_of_Experts(input, all_list_filters , list_num_experts,names)
    #------------------------------------------
    model = Model(inputs=[input], outputs=output)
    return model

In [ ]:
def model_2concat_gating(shape1, shape2 , all_list_filters_0, all_list_filters_1, list_num_experts_0, list_num_experts_1,
                         same_seed = [True,True], method = 'concat', num_classes=7):

    def set_seed(num_seed):
        SEED = num_seed
        np.random.seed(SEED)
        tf.random.set_seed(SEED)
        rn.seed(SEED)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        tf.compat.v1.keras.backend.set_session(sess)

    def expert_block(inputs , num_filters,  num_seed):
        set_seed(num_seed)
        conv = Conv2D(filters = num_filters[0],
                      kernel_size = (3,3) ,
                      activation = tf.nn.relu )(inputs)
        conv = MaxPooling2D(pool_size=(2, 2))(conv)
        conv = BatchNormalization()(conv)
        if  1 <= len(num_filters[1:]) <= 3  :
            for num_filter in num_filters[1:] :
                conv = Conv2D(filters = num_filter,
                              kernel_size = (3,3) ,
                              activation = tf.nn.relu,
                              padding = 'same')(conv)
                conv = MaxPooling2D(pool_size=(2, 2))(conv)
                conv = BatchNormalization()(conv)
        elif len(num_filters[1:]) >= 4 :
            for num_filter in num_filters[1:] :
                conv = Conv2D(filters = num_filter,
                              kernel_size = (3,3) ,
                              activation = tf.nn.relu,
                              padding = 'same')(conv)
                conv = BatchNormalization()(conv)

        #--------------------------------------------------
        conv = Flatten()(conv)
        conv = Dense(units =  1000,
                     activation = tf.nn.relu)(conv)
        conv = BatchNormalization()(conv)
        #-----------------------------
        conv = Dense(units =  600,
                     activation = tf.nn.relu)(conv)
        conv = BatchNormalization()(conv)
        conv = Dropout(rate = 0.5)(conv)
        y_hat = Dense(units = num_classes,
                      activation = None)(conv)
        return y_hat

    def Mixture_of_Experts(inputs,list_num_filters , num_experts, names, same_seed) :
        assert len(list_num_filters) == num_experts
        num_seed = 0
        expert_outs = []
        for i in range(num_experts):
            if same_seed :
                expert_network = expert_block(inputs, list_num_filters[i], num_seed )
            else :
                expert_network = expert_block(inputs, list_num_filters[i], i )
            expert_outs.append(expert_network)
        expert_concat = tf.keras.layers.Lambda(lambda x: tf.stack(x, axis=1))(expert_outs)
        print('expert concat shape : ', expert_concat.shape)
        # gating network
        gating = Conv2D(filters = 30,
                      kernel_size = (3,3) ,
                      activation = tf.nn.relu )(inputs)
        gating = MaxPooling2D(pool_size=(2, 2))(gating)
        gating = BatchNormalization()(gating)
        gating = Flatten()(gating)
        # ------------------------------------------------
        gating = Dense(800, activation = tf.nn.relu)(gating)
        gating = BatchNormalization()(gating)
        gating = Dropout(rate = 0.5)(gating)
        # -----------------
        gating = Dense(1200, activation = tf.nn.relu)(gating)
        gating = BatchNormalization()(gating)
        gating = Dropout(rate = 0.5)(gating)
        # -----------------
        gating = Dense(num_experts, activation = tf.nn.softmax)(gating)
        gating = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(gating)
        gate_mul_expert = tf.keras.layers.Lambda(lambda x: tf.math.reduce_sum(x[0] * x[1], axis=1),
                                                 name='gate_mul_expert_' + names )([expert_concat, gating])
        print('gate mul expert : ', gate_mul_expert.shape)

        return gate_mul_expert
    # ----------------------------------------------------------------------------------------------------------------------------------
    assert (len(all_list_filters_0) + len(all_list_filters_1)) == list_num_experts_0 + list_num_experts_1
    assert len(all_list_filters_0) == list_num_experts_0
    assert len(all_list_filters_1) == list_num_experts_1
    names = ['1','2']
    set_seed(0)
    #------------------------------------------
    input1 = Input(shape = (shape1, shape2, 1))
    MoE_1 = Mixture_of_Experts(input1, all_list_filters_0 , list_num_experts_0,names[0],same_seed[0])
    input2 = Input(shape = (shape1, shape2, 1))
    MoE_2 = Mixture_of_Experts(input2, all_list_filters_1 , list_num_experts_1,names[1],same_seed[1])
    #------------------------------------------
    if method == 'concat':
        concat = Concatenate(axis=1)([MoE_1, MoE_2])
        concat = Dense(units = 600,
                       activation = tf.nn.relu)(concat)
        concat = BatchNormalization()(concat)
        concat = Dropout(0.5)(concat)
        concat = Dense(units = 1000 ,
                       activation = tf.nn.relu)(concat)
        concat = BatchNormalization()(concat)
        concat = Dropout(0.5)(concat)
        softmax = Dense(12, activation = tf.nn.softmax )(concat)
    elif method == 'max':
        maxs = Maximum()([MoE_1,MoE_2])
        softmax = Softmax()(maxs)
    elif method == 'average':
        average = Average()([MoE_1,MoE_2])
        softmax = Softmax()(average)
    else :
        print('Wrong method')
        return False
    #------------------------------------------
    model = Model(inputs=[input1,input2], outputs=softmax)
    return model

In [ ]:
def model_Expert( shape1, shape2, num_seed, num_filters, num_classes=7) :
    def set_seed(num_seed):
        SEED = num_seed
        os.environ['PYTHONHASHSEED']=str(SEED)
        np.random.seed(SEED)
        tf.random.set_seed(SEED)
        rn.seed(SEED)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        tf.compat.v1.keras.backend.set_session(sess)



    inputs = Input(shape = (shape1, shape2, 1))
    set_seed(num_seed)
    conv = Conv2D(filters = num_filters[0],
                    kernel_size = (3,3) ,
                    activation = tf.nn.relu )(inputs)
    conv = MaxPooling2D(pool_size=(2, 2))(conv)
    conv = BatchNormalization()(conv)
    if  1 <= len(num_filters[1:]) <= 3  :
        for num_filter in num_filters[1:] :
            conv = Conv2D(filters = num_filter,
                            kernel_size = (3,3) ,
                            activation = tf.nn.relu,
                            padding = 'same')(conv)
            conv = MaxPooling2D(pool_size=(2, 2))(conv)
            conv = BatchNormalization()(conv)
    elif len(num_filters[1:]) >= 4 :
        for num_filter in num_filters[1:] :
            conv = Conv2D(filters = num_filter,
                            kernel_size = (3,3) ,
                            activation = tf.nn.relu,
                            padding = 'same')(conv)
            conv = BatchNormalization()(conv)
    else :
        pass
    # ----------------------------
    conv = Flatten()(conv)
    conv = Dense(units =  1000,
                    activation = tf.nn.relu)(conv)
    conv = BatchNormalization()(conv)
    #-----------------------------
    conv = Dense(units =  600,
                    activation = tf.nn.relu)(conv)
    conv = BatchNormalization()(conv)
    conv = Dropout(rate = 0.5)(conv)
    y_hat = Dense(units = num_classes,
                  activation=tf.nn.softmax)(conv)

    model = Model(inputs= inputs, outputs=y_hat)

    return model

# 7 labels

In [ ]:
def process_image(image):
  image = image/255.0
  return image

In [ ]:
def one_hot_encode_label(label):
  label = tf.cast(label, dtype=tf.int32)
  return tf.one_hot(label, depth=7)

In [ ]:
num_classes = 7

In [ ]:
train_ds = load_dataset("train")
val_ds = load_dataset("val")
test_ds = load_dataset("test")

train_size = len(train_ds)
val_size = len(val_ds)
test_size = len(test_ds)
print(f'Train Size: {train_size}\nValidation Size: {val_size}\nTest Size: {test_size}')

train_ds = train_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))
val_ds = val_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))
test_ds = test_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)

Train Size: 65399
Validation Size: 26174
Test Size: 17182


## IMG1

### Each Experts

#### Expert 1

In [ ]:
expert_1 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32])

expert_1.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score(num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_1.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_1.load_weights("img_1/expert_1.hdf5")
print(expert_1.evaluate(val_ds))
print(expert_1.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.2716 - categorical_accuracy: 0.4264 - precision: 0.4737 - recall: 0.3779 - f1_score: 0.4252
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_1.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


128/128 [==============================] - 27s 142ms/step - loss: 2.2716 - categorical_accuracy: 0.4264 - precision: 0.4737 - recall: 0.3779 - f1_score: 0.4252 - val_loss: 2.8960 - val_categorical_accuracy: 0.1534 - val_precision: 0.1534 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 1.0070 - categorical_accuracy: 0.7065 - precision: 0.7729 - recall: 0.6518 - f1_score: 0.7055
Epoch 2: val_f1_score improved from 0.04080 to 0.06655, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_1.hdf5
128/128 [==============================] - 3s 21ms/step - loss: 1.0086 - categorical_accuracy: 0.7057 - precision: 0.7721 - recall: 0.6509 - f1_score: 0.7046 - val_loss: 2.7020 - val_categorical_accuracy: 0.1674 - val_precision: 0.1632 - val_recall: 0.1603 - val_f1_score: 0.0665
Epoch 3/30
125/128 [============================>.] - ETA: 0s - loss: 0.5876 - categorical_accuracy: 0.8321 - precision: 0.8854 - recall: 0.796

#### Expert 2

In [ ]:
expert_2 = model_Expert( 32, 32, num_seed = 1,
                        num_filters = [32])
expert_2.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_2.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_2 = expert_2.fit(train_ds,
                      epochs = 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_2.load_weights("img_1/expert_2.hdf5")
print(expert_2.evaluate(val_ds))
print(expert_2.evaluate(test_ds))

Epoch 1/30
125/128 [============================>.] - ETA: 0s - loss: 2.2513 - categorical_accuracy: 0.4370 - precision: 0.4780 - recall: 0.3853 - f1_score: 0.4360
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_2.hdf5
128/128 [==============================] - 6s 28ms/step - loss: 2.2397 - categorical_accuracy: 0.4372 - precision: 0.4781 - recall: 0.3853 - f1_score: 0.4360 - val_loss: 2.5661 - val_categorical_accuracy: 0.1534 - val_precision: 0.1556 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
125/128 [============================>.] - ETA: 0s - loss: 0.9543 - categorical_accuracy: 0.7221 - precision: 0.7826 - recall: 0.6737 - f1_score: 0.7210
Epoch 2: val_f1_score improved from 0.04080 to 0.17834, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_2.hdf5
128/128 [==============================] - 2s 19ms/step - loss: 0.9497 - categorical_accuracy: 0.7229 - precision: 0.7831 - recall: 0

#### Expert 3

In [ ]:
expert_3 = model_Expert( 32, 32, num_seed = 2,
                        num_filters = [32])
expert_3.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_3.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_3 = expert_3.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_3.load_weights("img_1/expert_3.hdf5")
print(expert_3.evaluate(val_ds))
print(expert_3.evaluate(test_ds))

Epoch 1/30
125/128 [============================>.] - ETA: 0s - loss: 2.2732 - categorical_accuracy: 0.4290 - precision: 0.4745 - recall: 0.3793 - f1_score: 0.4286
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_3.hdf5
128/128 [==============================] - 6s 28ms/step - loss: 2.2619 - categorical_accuracy: 0.4308 - precision: 0.4762 - recall: 0.3812 - f1_score: 0.4305 - val_loss: 2.4267 - val_categorical_accuracy: 0.1534 - val_precision: 0.1534 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.9671 - categorical_accuracy: 0.7144 - precision: 0.7783 - recall: 0.6609 - f1_score: 0.7134
Epoch 2: val_f1_score improved from 0.04080 to 0.06459, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_3.hdf5
128/128 [==============================] - 2s 19ms/step - loss: 0.9671 - categorical_accuracy: 0.7142 - precision: 0.7782 - recall: 0

#### Expert 4

In [ ]:
expert_4 = model_Expert( 32, 32, num_seed = 3,
                        num_filters = [32])
expert_4.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_4.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_4 = expert_4.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_4.load_weights("img_1/expert_4.hdf5")
print(expert_4.evaluate(val_ds))
print(expert_4.evaluate(test_ds))

Epoch 1/30
125/128 [============================>.] - ETA: 0s - loss: 2.3016 - categorical_accuracy: 0.4207 - precision: 0.4659 - recall: 0.3695 - f1_score: 0.4199
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_4.hdf5
128/128 [==============================] - 5s 22ms/step - loss: 2.2938 - categorical_accuracy: 0.4216 - precision: 0.4670 - recall: 0.3705 - f1_score: 0.4207 - val_loss: 2.4616 - val_categorical_accuracy: 0.1534 - val_precision: 0.1592 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
125/128 [============================>.] - ETA: 0s - loss: 1.0182 - categorical_accuracy: 0.7034 - precision: 0.7701 - recall: 0.6518 - f1_score: 0.7019
Epoch 2: val_f1_score improved from 0.04080 to 0.04627, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_4.hdf5
128/128 [==============================] - 2s 20ms/step - loss: 1.0116 - categorical_accuracy: 0.7049 - precision: 0.7712 - recall: 0

#### Expert 5

In [ ]:
expert_5 = model_Expert( 32, 32, num_seed = 4,
                        num_filters = [32])
expert_5.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_5.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_5 = expert_5.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )

print("best model: ")
expert_5.load_weights("img_1/expert_5.hdf5")
print(expert_5.evaluate(val_ds))
print(expert_5.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.2688 - categorical_accuracy: 0.4229 - precision: 0.4647 - recall: 0.3659 - f1_score: 0.4216
Epoch 1: val_f1_score improved from -inf to 0.07730, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_5.hdf5
128/128 [==============================] - 5s 23ms/step - loss: 2.2688 - categorical_accuracy: 0.4229 - precision: 0.4647 - recall: 0.3659 - f1_score: 0.4216 - val_loss: 2.3416 - val_categorical_accuracy: 0.1730 - val_precision: 0.2845 - val_recall: 0.1331 - val_f1_score: 0.0773
Epoch 2/30
125/128 [============================>.] - ETA: 0s - loss: 1.0226 - categorical_accuracy: 0.6981 - precision: 0.7682 - recall: 0.6432 - f1_score: 0.6970
Epoch 2: val_f1_score improved from 0.07730 to 0.10921, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_5.hdf5
128/128 [==============================] - 3s 20ms/step - loss: 1.0163 - categorical_accuracy: 0.6990 - precision: 0.7688 - recall: 0

### Combining Experts

In [ ]:
img1_gating = model_1concat_gating(shape1 = 32,
                                   shape2 = 32,
                                       all_list_filters = [[32],
                                                           [32],
                                                           [32],
                                                           [32],
                                                           [32]
                                                          ],
                                       list_num_experts = 5,
                                       same_seed = False)
img1_gating.summary()

expert concat shape :  (None, 5, 7)
gate mul expert :  (None, 7)
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 conv2d_10 (Conv2D)          (None, 30, 30, 30)           300       ['input_6[0][0]']             
                                                                                                  
 conv2d_5 (Conv2D)           (None, 30, 30, 32)           320       ['input_6[0][0]']             
                                                                                                  
 conv2d_6 (Conv2D)           (None, 30, 30, 32)           320       ['input_6[0][0]']             
                           

In [ ]:
img1_gating.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )

f1_callback_concat = ModelCheckpoint("img_1/raw_img1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = img1_gating.fit(train_ds,
                      epochs= 10,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
img1_gating.load_weights("img_1/raw_img1.hdf5")
print(img1_gating.evaluate(val_ds))
print(img1_gating.evaluate(test_ds))

Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 1.9114 - categorical_accuracy: 0.3789 - precision: 0.4785 - recall: 0.2460 - f1_score: 0.3777
Epoch 1: val_f1_score improved from -inf to 0.04088, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/raw_img1.hdf5
128/128 [==============================] - 31s 141ms/step - loss: 1.9114 - categorical_accuracy: 0.3789 - precision: 0.4785 - recall: 0.2460 - f1_score: 0.3777 - val_loss: 1.8749 - val_categorical_accuracy: 0.1534 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0409
Epoch 2/10
128/128 [==============================] - ETA: 0s - loss: 0.6379 - categorical_accuracy: 0.8159 - precision: 0.8875 - recall: 0.7451 - f1_score: 0.8154
Epoch 2: val_f1_score improved from 0.04088 to 0.13690, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/raw_img1.hdf5
128/128 [==============================] - 13s 104ms/step - loss: 0.6379 - categorical_accuracy: 0.8159 - precision: 0.8875

# IMG2

### Each Experts

##### Expert 1

In [ ]:
expert_1 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32])
expert_1.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_1.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )

print("best model: ")
expert_1.load_weights("img_2/expert_1.hdf5")
print(expert_1.evaluate(val_ds))
print(expert_1.evaluate(test_ds))

Epoch 1/30
125/128 [============================>.] - ETA: 0s - loss: 2.3156 - categorical_accuracy: 0.4174 - precision: 0.4644 - recall: 0.3712 - f1_score: 0.4162
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_1.hdf5
128/128 [==============================] - 5s 23ms/step - loss: 2.3037 - categorical_accuracy: 0.4189 - precision: 0.4656 - recall: 0.3722 - f1_score: 0.4177 - val_loss: 3.3545 - val_categorical_accuracy: 0.1534 - val_precision: 0.1534 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 1.0005 - categorical_accuracy: 0.7128 - precision: 0.7780 - recall: 0.6606 - f1_score: 0.7121
Epoch 2: val_f1_score did not improve from 0.04080
128/128 [==============================] - 3s 20ms/step - loss: 1.0006 - categorical_accuracy: 0.7120 - precision: 0.7771 - recall: 0.6593 - f1_score: 0.7112 - val_loss: 3.6131 - val_categorical_accuracy: 0.1534 -

#### Expert 2

In [ ]:
expert_2 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28])
expert_2.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )




f1_callback_concat = ModelCheckpoint("img_2/expert_2.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_2.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_2.load_weights("img_2/expert_2.hdf5")
print(expert_2.evaluate(val_ds))
print(expert_2.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.2123 - categorical_accuracy: 0.4279 - precision: 0.4770 - recall: 0.3721 - f1_score: 0.4271
Epoch 1: val_f1_score improved from -inf to 0.03336, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_2.hdf5
128/128 [==============================] - 6s 23ms/step - loss: 2.2123 - categorical_accuracy: 0.4279 - precision: 0.4770 - recall: 0.3721 - f1_score: 0.4271 - val_loss: 2.1903 - val_categorical_accuracy: 0.1378 - val_precision: 0.6301 - val_recall: 0.1034 - val_f1_score: 0.0334
Epoch 2/30
124/128 [============================>.] - ETA: 0s - loss: 0.9109 - categorical_accuracy: 0.7333 - precision: 0.7946 - recall: 0.6855 - f1_score: 0.7320
Epoch 2: val_f1_score did not improve from 0.03336
128/128 [==============================] - 2s 17ms/step - loss: 0.9133 - categorical_accuracy: 0.7322 - precision: 0.7935 - recall: 0.6843 - f1_score: 0.7310 - val_loss: 3.3331 - val_categorical_accuracy: 0.1378 -

#### Expert 3

In [ ]:
expert_3 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24])
expert_3.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_3.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_3.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_3.load_weights("img_2/expert_3.hdf5")
print(expert_3.evaluate(val_ds))
print(expert_3.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.3591 - categorical_accuracy: 0.3840 - precision: 0.4338 - recall: 0.3260 - f1_score: 0.3826
Epoch 1: val_f1_score improved from -inf to 0.10551, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_3.hdf5
128/128 [==============================] - 7s 20ms/step - loss: 2.3591 - categorical_accuracy: 0.3840 - precision: 0.4338 - recall: 0.3260 - f1_score: 0.3826 - val_loss: 2.0500 - val_categorical_accuracy: 0.1827 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.1055
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 1.1220 - categorical_accuracy: 0.6682 - precision: 0.7368 - recall: 0.6091 - f1_score: 0.6668
Epoch 2: val_f1_score did not improve from 0.10551
128/128 [==============================] - 2s 15ms/step - loss: 1.1230 - categorical_accuracy: 0.6674 - precision: 0.7360 - recall: 0.6081 - f1_score: 0.6659 - val_loss: 2.5358 - val_categorical_accuracy: 

#### Expert 4

In [ ]:
expert_4 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24])
expert_4.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_4.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_4.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_4.load_weights("img_2/expert_4.hdf5")
print(expert_4.evaluate(val_ds))
print(expert_4.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.8055 - categorical_accuracy: 0.2703 - precision: 0.2988 - recall: 0.2045 - f1_score: 0.2690
Epoch 1: val_f1_score improved from -inf to 0.03334, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_4.hdf5
128/128 [==============================] - 7s 24ms/step - loss: 2.8055 - categorical_accuracy: 0.2703 - precision: 0.2988 - recall: 0.2045 - f1_score: 0.2690 - val_loss: 2.1838 - val_categorical_accuracy: 0.1370 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0333
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 1.9240 - categorical_accuracy: 0.4214 - precision: 0.4865 - recall: 0.3334 - f1_score: 0.4189
Epoch 2: val_f1_score did not improve from 0.03334
128/128 [==============================] - 2s 16ms/step - loss: 1.9265 - categorical_accuracy: 0.4207 - precision: 0.4855 - recall: 0.3330 - f1_score: 0.4179 - val_loss: 3.3602 - val_categorical_accuracy: 

#### Expert 5

In [ ]:
expert_5 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24,20])
expert_5.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_5.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_5.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )

print("best model: ")
expert_5.load_weights("img_2/expert_5.hdf5")
print(expert_5.evaluate(val_ds))
print(expert_5.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.2170 - categorical_accuracy: 0.4402 - precision: 0.4883 - recall: 0.3910 - f1_score: 0.4388
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_5.hdf5
128/128 [==============================] - 9s 37ms/step - loss: 2.2170 - categorical_accuracy: 0.4402 - precision: 0.4883 - recall: 0.3910 - f1_score: 0.4388 - val_loss: 3.2915 - val_categorical_accuracy: 0.1534 - val_precision: 0.1534 - val_recall: 0.1534 - val_f1_score: 0.0408
Epoch 2/30
128/128 [==============================] - ETA: 0s - loss: 0.8339 - categorical_accuracy: 0.7682 - precision: 0.8207 - recall: 0.7298 - f1_score: 0.7664
Epoch 2: val_f1_score improved from 0.04080 to 0.12553, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_5.hdf5
128/128 [==============================] - 4s 28ms/step - loss: 0.8339 - categorical_accuracy: 0.7682 - precision: 0.8207 - recall: 0

#### Expert 6

In [ ]:
expert_6 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24,20,20])
expert_6.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_6.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_6.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_6.load_weights("img_2/expert_6.hdf5")
print(expert_6.evaluate(val_ds))
print(expert_6.evaluate(test_ds))

Epoch 1/30
128/128 [==============================] - ETA: 0s - loss: 2.2392 - categorical_accuracy: 0.4443 - precision: 0.4906 - recall: 0.3972 - f1_score: 0.4429
Epoch 1: val_f1_score improved from -inf to 0.04080, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_6.hdf5
128/128 [==============================] - 9s 36ms/step - loss: 2.2392 - categorical_accuracy: 0.4443 - precision: 0.4906 - recall: 0.3972 - f1_score: 0.4429 - val_loss: 2.7055 - val_categorical_accuracy: 0.1534 - val_precision: 0.2269 - val_recall: 0.1375 - val_f1_score: 0.0408
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.8480 - categorical_accuracy: 0.7633 - precision: 0.8136 - recall: 0.7263 - f1_score: 0.7617
Epoch 2: val_f1_score improved from 0.04080 to 0.14517, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_6.hdf5
128/128 [==============================] - 4s 35ms/step - loss: 0.8464 - categorical_accuracy: 0.7634 - precision: 0.8137 - recall: 0

### Combining Experts

In [ ]:
img2_gating = model_1concat_gating(shape1 = 32,
                                       shape2 = 32,
                                       all_list_filters = [[32],[32,28],[32,28,24],[32,28,24,24],
                                                           [32,28,24,24,20],
                                                           [32,28,24,24,20,20]
                                                          ],
                                       list_num_experts = 6,
                                       same_seed = False)


img2_gating.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )

f1_callback_concat = ModelCheckpoint("raw_img2x.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_2 = img2_gating.fit(train_ds,
                      epochs= 10,
                      validation_data = val_ds,
                      callbacks = [f1_callback_concat]
                     )

print("best model: ")
img2_gating.load_weights("raw_img2x.hdf5")

print(img2_gating.evaluate(val_ds))
print(img2_gating.evaluate(test_ds))

expert concat shape :  (None, 6, 7)
gate mul expert :  (None, 7)
Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 2.0612 - categorical_accuracy: 0.3102 - precision: 0.3937 - recall: 0.1640 - f1_score: 0.3078
Epoch 1: val_f1_score improved from -inf to 0.09847, saving model to /content/drive/MyDrive/Colab Notebooks/raw_img2x.hdf5
128/128 [==============================] - 39s 137ms/step - loss: 2.0612 - categorical_accuracy: 0.3102 - precision: 0.3937 - recall: 0.1640 - f1_score: 0.3078 - val_loss: 1.9575 - val_categorical_accuracy: 0.1939 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0985
Epoch 2/10
128/128 [==============================] - ETA: 0s - loss: 0.9431 - categorical_accuracy: 0.6995 - precision: 0.8177 - recall: 0.5749 - f1_score: 0.6980
Epoch 2: val_f1_score did not improve from 0.09847
128/128 [==============================] - 14s 112ms/step - loss: 0.9431 - categorical_accuracy: 0.6995 - precision: 0.8177 - recall: 0.5749 - 

# 2 labels

In [ ]:
def process_image(image):
  image = image/255.0
  return image

In [ ]:
def one_hot_encode_label(label):
  label = tf.cast(label, dtype=tf.int32)
  label = int(label<=3)
  return tf.one_hot(label, depth=2)

In [ ]:
num_classes = 2

In [ ]:
train_ds = load_dataset("train")
val_ds = load_dataset("val")
test_ds = load_dataset("test")

train_size = len(train_ds)
val_size = len(val_ds)
test_size = len(test_ds)
print(f'Train Size: {train_size}\nValidation Size: {val_size}\nTest Size: {test_size}')

train_ds = train_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))
val_ds = val_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))
test_ds = test_ds.map(lambda x, y: (process_image(x), one_hot_encode_label(y)))

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().batch(batch_size=2**9).prefetch(buffer_size=AUTOTUNE)

Train Size: 65399
Validation Size: 26174
Test Size: 17182


## IMG1

### Each Experts

#### Expert 1

In [ ]:
expert_1 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32], num_classes=2)

expert_1.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score(num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_1.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = [
                                    f1_callback_concat,
                                  ],
                     )


print("best model: ")
expert_1.load_weights("img_1/expert_1.hdf5")
print(expert_1.evaluate(val_ds))
print(expert_1.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 1.1752 - categorical_accuracy: 0.6902 - precision: 0.6902 - recall: 0.6902 - f1_score: 0.6915
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_1.hdf5
128/128 [==============================] - 16s 66ms/step - loss: 1.1706 - categorical_accuracy: 0.6913 - precision: 0.6913 - recall: 0.6913 - f1_score: 0.6925 - val_loss: 2.4298 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 0.5639 - categorical_accuracy: 0.7762 - precision: 0.7762 - recall: 0.7762 - f1_score: 0.7763
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 3s 20ms/step - loss: 0.5603 - categorical_accuracy: 0.7776 - precision: 0.7776 - recall: 0.7776 - f1_score: 0.7778 - val_loss: 2.2067 - val_categorical_accuracy: 0.5760 

#### Expert 2

In [ ]:
expert_2 = model_Expert( 32, 32, num_seed = 1,
                        num_filters = [32], num_classes=2)
expert_2.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_2.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_2 = expert_2.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = [
                                    f1_callback_concat,
                                  ],
                     )


print("best model: ")
expert_2.load_weights("img_1/expert_2.hdf5")
print(expert_2.evaluate(val_ds))
print(expert_2.evaluate(test_ds))

Epoch 1/30
126/128 [============================>.] - ETA: 0s - loss: 1.0565 - categorical_accuracy: 0.7147 - precision: 0.7147 - recall: 0.7147 - f1_score: 0.7160
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_2.hdf5
128/128 [==============================] - 5s 24ms/step - loss: 1.0457 - categorical_accuracy: 0.7172 - precision: 0.7172 - recall: 0.7172 - f1_score: 0.7185 - val_loss: 1.8589 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
128/128 [==============================] - ETA: 0s - loss: 0.5627 - categorical_accuracy: 0.7711 - precision: 0.7711 - recall: 0.7711 - f1_score: 0.7710
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 3s 23ms/step - loss: 0.5627 - categorical_accuracy: 0.7711 - precision: 0.7711 - recall: 0.7711 - f1_score: 0.7710 - val_loss: 2.1225 - val_categorical_accuracy: 0.5760 -

#### Expert 3

In [ ]:
expert_3 = model_Expert( 32, 32, num_seed = 2,
                        num_filters = [32], num_classes=2)
expert_3.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_3.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_3 = expert_3.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = [
                                    f1_callback_concat,
                                  ],
                     )


print("best model: ")
expert_3.load_weights("img_1/expert_3.hdf5")
print(expert_3.evaluate(val_ds))
print(expert_3.evaluate(test_ds))

Epoch 1/30
125/128 [============================>.] - ETA: 0s - loss: 1.0678 - categorical_accuracy: 0.7009 - precision: 0.7009 - recall: 0.7009 - f1_score: 0.7022
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_3.hdf5
128/128 [==============================] - 6s 29ms/step - loss: 1.0509 - categorical_accuracy: 0.7058 - precision: 0.7058 - recall: 0.7058 - f1_score: 0.7070 - val_loss: 1.5008 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.5262 - categorical_accuracy: 0.7786 - precision: 0.7786 - recall: 0.7786 - f1_score: 0.7787
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 2s 18ms/step - loss: 0.5244 - categorical_accuracy: 0.7794 - precision: 0.7794 - recall: 0.7794 - f1_score: 0.7795 - val_loss: 1.9299 - val_categorical_accuracy: 0.5760 -

#### Expert 4

In [ ]:
expert_4 = model_Expert( 32, 32, num_seed = 3,
                        num_filters = [32], num_classes=2)
expert_4.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_4.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_4 = expert_4.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = [
                                    f1_callback_concat,
                                  ],
                     )


print("best model: ")
expert_4.load_weights("img_1/expert_4.hdf5")
print(expert_4.evaluate(val_ds))
print(expert_4.evaluate(test_ds))

Epoch 1/30
126/128 [============================>.] - ETA: 0s - loss: 1.0552 - categorical_accuracy: 0.7045 - precision: 0.7045 - recall: 0.7045 - f1_score: 0.7057
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_4.hdf5
128/128 [==============================] - 5s 22ms/step - loss: 1.0439 - categorical_accuracy: 0.7075 - precision: 0.7075 - recall: 0.7075 - f1_score: 0.7088 - val_loss: 1.8321 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.5458 - categorical_accuracy: 0.7766 - precision: 0.7766 - recall: 0.7766 - f1_score: 0.7765
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 2s 18ms/step - loss: 0.5439 - categorical_accuracy: 0.7773 - precision: 0.7773 - recall: 0.7773 - f1_score: 0.7773 - val_loss: 1.3030 - val_categorical_accuracy: 0.5760 -

#### Expert 5

In [ ]:
expert_5 = model_Expert( 32, 32, num_seed = 4,
                        num_filters = [32], num_classes=2)
expert_5.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_1/expert_5.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_5 = expert_5.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = [
                                    f1_callback_concat,
                                  ],
                     )


print("best model: ")
expert_5.load_weights("img_1/expert_5.hdf5")
print(expert_5.evaluate(val_ds))
print(expert_5.evaluate(test_ds))

Epoch 1/30
126/128 [============================>.] - ETA: 0s - loss: 1.0998 - categorical_accuracy: 0.6972 - precision: 0.6972 - recall: 0.6972 - f1_score: 0.6985
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/expert_5.hdf5
128/128 [==============================] - 7s 37ms/step - loss: 1.0885 - categorical_accuracy: 0.7001 - precision: 0.7001 - recall: 0.7001 - f1_score: 0.7014 - val_loss: 1.8041 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 0.5385 - categorical_accuracy: 0.7776 - precision: 0.7776 - recall: 0.7776 - f1_score: 0.7778
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 2s 18ms/step - loss: 0.5331 - categorical_accuracy: 0.7802 - precision: 0.7802 - recall: 0.7802 - f1_score: 0.7803 - val_loss: 1.3967 - val_categorical_accuracy: 0.5760 -

### Combining Experts

In [ ]:
img1_gating = model_1concat_gating(shape1 = 32,
                                   shape2 = 32,
                                       all_list_filters = [[32],
                                                           [32],
                                                           [32],
                                                           [32],
                                                           [32]
                                                          ],
                                       list_num_experts = 5,
                                       same_seed = False, num_classes=2)
img1_gating.summary()

expert concat shape :  (None, 5, 2)
gate mul expert :  (None, 2)
Model: "model_18"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_19 (InputLayer)       [(None, 32, 32, 1)]          0         []                            
                                                                                                  
 conv2d_64 (Conv2D)          (None, 30, 30, 30)           300       ['input_19[0][0]']            
                                                                                                  
 conv2d_59 (Conv2D)          (None, 30, 30, 32)           320       ['input_19[0][0]']            
                                                                                                  
 conv2d_60 (Conv2D)          (None, 30, 30, 32)           320       ['input_19[0][0]']            
                          

In [ ]:
img1_gating.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )

f1_callback_concat = ModelCheckpoint("img_1/raw_img1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = img1_gating.fit(train_ds,
                      epochs= 10,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
img1_gating.load_weights("img_1/raw_img1.hdf5")
print(img1_gating.evaluate(val_ds))
print(img1_gating.evaluate(test_ds))

Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 0.5823 - categorical_accuracy: 0.7621 - precision: 0.7621 - recall: 0.7621 - f1_score: 0.7633
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_1/raw_img1.hdf5
128/128 [==============================] - 36s 185ms/step - loss: 0.5823 - categorical_accuracy: 0.7621 - precision: 0.7621 - recall: 0.7621 - f1_score: 0.7633 - val_loss: 0.8113 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/10
128/128 [==============================] - ETA: 0s - loss: 0.1950 - categorical_accuracy: 0.9259 - precision: 0.9259 - recall: 0.9259 - f1_score: 0.9261
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 11s 87ms/step - loss: 0.1950 - categorical_accuracy: 0.9259 - precision: 0.9259 - recall: 0.9259 - f1_score: 0.9261 - val_loss: 0.9822 - val_categorical_accuracy: 0.576

# IMG2

### Each Experts

##### Expert 1

In [ ]:
expert_1 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32], num_classes = num_classes)
expert_1.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_1.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_1.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )

print("best model: ")
expert_1.load_weights("img_2/expert_1.hdf5")
print(expert_1.evaluate(val_ds))
print(expert_1.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 0.9116 - categorical_accuracy: 0.7376 - precision: 0.7376 - recall: 0.7376 - f1_score: 0.7389
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_1.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


128/128 [==============================] - 11s 40ms/step - loss: 0.9085 - categorical_accuracy: 0.7384 - precision: 0.7384 - recall: 0.7384 - f1_score: 0.7397 - val_loss: 1.2927 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
125/128 [============================>.] - ETA: 0s - loss: 0.5985 - categorical_accuracy: 0.7834 - precision: 0.7834 - recall: 0.7834 - f1_score: 0.7841
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 3s 26ms/step - loss: 0.5866 - categorical_accuracy: 0.7879 - precision: 0.7879 - recall: 0.7879 - f1_score: 0.7885 - val_loss: 1.5392 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 3/30
128/128 [==============================] - ETA: 0s - loss: 0.5120 - categorical_accuracy: 0.8022 - precision: 0.8022 - recall: 0.8022 - f1_score: 0.8028
Epoch 3: val_f1_score improved from 0.42098 to 0.46431, savi

#### Expert 2

In [ ]:
expert_2 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28], num_classes = num_classes)
expert_2.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )




f1_callback_concat = ModelCheckpoint("img_2/expert_2.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_2.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_2.load_weights("img_2/expert_2.hdf5")
print(expert_2.evaluate(val_ds))
print(expert_2.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 0.8947 - categorical_accuracy: 0.7345 - precision: 0.7345 - recall: 0.7345 - f1_score: 0.7357
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_2.hdf5
128/128 [==============================] - 5s 19ms/step - loss: 0.8911 - categorical_accuracy: 0.7355 - precision: 0.7355 - recall: 0.7355 - f1_score: 0.7367 - val_loss: 0.9062 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.6020 - categorical_accuracy: 0.7910 - precision: 0.7910 - recall: 0.7910 - f1_score: 0.7918
Epoch 2: val_f1_score improved from 0.42098 to 0.42185, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_2.hdf5
128/128 [==============================] - 3s 21ms/step - loss: 0.5989 - categorical_accuracy: 0.7922 - precision: 0.7922 - recall: 0

#### Expert 3

In [ ]:
expert_3 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24], num_classes = num_classes)
expert_3.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_3.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_3.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_3.load_weights("img_2/expert_3.hdf5")
print(expert_3.evaluate(val_ds))
print(expert_3.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 0.9504 - categorical_accuracy: 0.7054 - precision: 0.7054 - recall: 0.7054 - f1_score: 0.7067
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_3.hdf5
128/128 [==============================] - 6s 22ms/step - loss: 0.9483 - categorical_accuracy: 0.7061 - precision: 0.7061 - recall: 0.7061 - f1_score: 0.7074 - val_loss: 0.8080 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
126/128 [============================>.] - ETA: 0s - loss: 0.5957 - categorical_accuracy: 0.7799 - precision: 0.7799 - recall: 0.7799 - f1_score: 0.7807
Epoch 2: val_f1_score improved from 0.42098 to 0.43611, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_3.hdf5
128/128 [==============================] - 3s 21ms/step - loss: 0.5886 - categorical_accuracy: 0.7826 - precision: 0.7826 - recall: 0

#### Expert 4

In [ ]:
expert_4 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24], num_classes = num_classes)
expert_4.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_4.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_4.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_4.load_weights("img_2/expert_4.hdf5")
print(expert_4.evaluate(val_ds))
print(expert_4.evaluate(test_ds))

Epoch 1/30
126/128 [============================>.] - ETA: 0s - loss: 1.1070 - categorical_accuracy: 0.6446 - precision: 0.6446 - recall: 0.6446 - f1_score: 0.6463
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_4.hdf5
128/128 [==============================] - 6s 22ms/step - loss: 1.1052 - categorical_accuracy: 0.6444 - precision: 0.6444 - recall: 0.6444 - f1_score: 0.6462 - val_loss: 0.6680 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
125/128 [============================>.] - ETA: 0s - loss: 0.6894 - categorical_accuracy: 0.7223 - precision: 0.7223 - recall: 0.7223 - f1_score: 0.7230
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 3s 21ms/step - loss: 0.6783 - categorical_accuracy: 0.7271 - precision: 0.7271 - recall: 0.7271 - f1_score: 0.7278 - val_loss: 0.8089 - val_categorical_accuracy: 0.4372 -

#### Expert 5

In [ ]:
expert_5 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24,20], num_classes = num_classes)
expert_5.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_5.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_5.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )

print("best model: ")
expert_5.load_weights("img_2/expert_5.hdf5")
print(expert_5.evaluate(val_ds))
print(expert_5.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 0.9304 - categorical_accuracy: 0.7230 - precision: 0.7230 - recall: 0.7230 - f1_score: 0.7242
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_5.hdf5
128/128 [==============================] - 9s 40ms/step - loss: 0.9262 - categorical_accuracy: 0.7242 - precision: 0.7242 - recall: 0.7242 - f1_score: 0.7255 - val_loss: 1.1509 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
128/128 [==============================] - ETA: 0s - loss: 0.5875 - categorical_accuracy: 0.7930 - precision: 0.7930 - recall: 0.7930 - f1_score: 0.7936
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 3s 26ms/step - loss: 0.5875 - categorical_accuracy: 0.7930 - precision: 0.7930 - recall: 0.7930 - f1_score: 0.7936 - val_loss: 1.2215 - val_categorical_accuracy: 0.5760 -

#### Expert 6

In [ ]:
expert_6 = model_Expert( 32, 32, num_seed = 0,
                        num_filters = [32,28,24,24,20,20], num_classes = num_classes)
expert_6.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )



f1_callback_concat = ModelCheckpoint("img_2/expert_6.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_1 = expert_6.fit(train_ds,
                      epochs= 30,
                      validation_data = val_ds,
                      callbacks = f1_callback_concat
                     )


print("best model: ")
expert_6.load_weights("img_2/expert_6.hdf5")
print(expert_6.evaluate(val_ds))
print(expert_6.evaluate(test_ds))

Epoch 1/30
127/128 [============================>.] - ETA: 0s - loss: 0.9128 - categorical_accuracy: 0.7187 - precision: 0.7187 - recall: 0.7187 - f1_score: 0.7199
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/img_2/expert_6.hdf5
128/128 [==============================] - 10s 43ms/step - loss: 0.9093 - categorical_accuracy: 0.7196 - precision: 0.7196 - recall: 0.7196 - f1_score: 0.7208 - val_loss: 0.8711 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/30
127/128 [============================>.] - ETA: 0s - loss: 0.5582 - categorical_accuracy: 0.7843 - precision: 0.7843 - recall: 0.7843 - f1_score: 0.7851
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 4s 32ms/step - loss: 0.5559 - categorical_accuracy: 0.7852 - precision: 0.7852 - recall: 0.7852 - f1_score: 0.7860 - val_loss: 1.5613 - val_categorical_accuracy: 0.5760 

### Combining Experts

In [ ]:
img2_gating = model_1concat_gating(shape1 = 32,
                                       shape2 = 32,
                                       all_list_filters = [[32],[32,28],[32,28,24],[32,28,24,24],
                                                           [32,28,24,24,20],
                                                           [32,28,24,24,20,20]
                                                          ],
                                       list_num_experts = 6,
                                       same_seed = True,
                                   num_classes=num_classes)


img2_gating.compile(
                 optimizer = tf.keras.optimizers.Adadelta(learning_rate= 1e-1, rho=0.95, epsilon=1e-08, weight_decay=0.0),
                 loss = keras.losses.CategoricalCrossentropy(name='categorical_crossentropy'),
                 metrics = [
                            keras.metrics.CategoricalAccuracy(name='categorical_accuracy'),
                            keras.metrics.Precision(name='precision'),
                            keras.metrics.Recall(name='recall'),
                            tfa.metrics.F1Score( num_classes=num_classes,
                                                name = 'f1_score',
                                                average='weighted')
                           ]
    )

f1_callback_concat = ModelCheckpoint("raw_img2x.hdf5",
                              monitor='val_f1_score',
                              verbose=1,
                              save_best_only=True,
                              mode='max')
history_2 = img2_gating.fit(train_ds,
                      epochs= 10,
                      validation_data = val_ds,
                      callbacks = [f1_callback_concat]
                     )

print("best model: ")
img2_gating.load_weights("raw_img2x.hdf5")

print(img2_gating.evaluate(val_ds))
print(img2_gating.evaluate(test_ds))

expert concat shape :  (None, 6, 2)
gate mul expert :  (None, 2)
Epoch 1/10
128/128 [==============================] - ETA: 0s - loss: 0.6424 - categorical_accuracy: 0.7239 - precision: 0.7239 - recall: 0.7239 - f1_score: 0.7252
Epoch 1: val_f1_score improved from -inf to 0.42098, saving model to /content/drive/MyDrive/Colab Notebooks/raw_img2x.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


128/128 [==============================] - 55s 165ms/step - loss: 0.6424 - categorical_accuracy: 0.7239 - precision: 0.7239 - recall: 0.7239 - f1_score: 0.7252 - val_loss: 0.8112 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 2/10
128/128 [==============================] - ETA: 0s - loss: 0.2566 - categorical_accuracy: 0.8993 - precision: 0.8993 - recall: 0.8993 - f1_score: 0.8994
Epoch 2: val_f1_score did not improve from 0.42098
128/128 [==============================] - 14s 111ms/step - loss: 0.2566 - categorical_accuracy: 0.8993 - precision: 0.8993 - recall: 0.8993 - f1_score: 0.8994 - val_loss: 1.0547 - val_categorical_accuracy: 0.5760 - val_precision: 0.5760 - val_recall: 0.5760 - val_f1_score: 0.4210
Epoch 3/10
128/128 [==============================] - ETA: 0s - loss: 0.1232 - categorical_accuracy: 0.9555 - precision: 0.9555 - recall: 0.9555 - f1_score: 0.9555
Epoch 3: val_f1_score improved from 0.42098 to 0.58932, s